In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score


### Séparation train/test

**Mais avant de faire l'imputation, on fait la séparation des ensembles d'entraînement et de test**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_team, train_target['HOME_WINS'], test_size=0.2, random_state=42)

In [ ]:
print(f"Nombre de données d'entrainement: {X_train.shape[0]}")
print(f"Nombre de données de test: {X_test.shape[0]}")

### Imputation des données

On calcule la médiane des colonnes dans l'ensemble d'entraînement et on l'utilise pour imputer les données manquantes dans l'ensemble d'entraînement et de test

In [ ]:
for column in column_missing_values[column_missing_values > 0].index:
    _column_median = X_train[column].median()

    if X_train[column].isna().sum() > 0:
        X_train[column] = X_train[column].fillna(_column_median)

    if X_test[column].isna().sum() > 0:
        X_test[column] = X_test[column].fillna(_column_median)

In [ ]:
print(f'Nombre de valeurs manquantes dans le train: {X_train.isna().sum().sum()}')
print(f'Nombre de valeurs manquantes dans le test: {X_test.isna().sum().sum()}')

## Classification Tree

Cette partie n'est pas encore fini, ne pas considerer

In [ ]:
my_kfold = KFold(n_splits=5, shuffle=True, random_state=42)


parameters = {'n_estimators': range(5, 100, 20)}

firstRF = GridSearchCV(estimator=RandomForestRegressor(random_state=42), param_grid=parameters, n_jobs=-1, cv=my_kfold)

firstRF.fit(X_train, y_train)
cv_score = firstRF.cv_results_['mean_test_score']

In [ ]:
firstRF.score(X_test, y_test)

In [ ]:
rf = RandomForestClassifier(n_estimators=20, max_features=4, random_state=42).fit(X_train, y_train)

In [ ]:
rf.score(X_test, y_test)

## Submission

In [ ]:
path_team_away_test = f'{base_path}/Test_Data/test_away_team_statistics_df.csv'
path_team_home_test = f'{base_path}/Test_Data/test_home_team_statistics_df.csv'

test_team_home = pd.read_csv(path_team_home_test, sep=',')
test_team_away = pd.read_csv(path_team_away_test, sep=',')

test_team_home.columns = 'HOME_' + test_team_home.columns
test_team_away.columns = 'AWAY_' + test_team_away.columns

test_team =  pd.concat([test_team_home, test_team_away.iloc[:, 1:]],join='inner',axis=1)
test_team = test_team.rename(columns={'HOME_ID':'ID'})

In [ ]:
test_team

In [ ]:
predictions = rf.predict(test_team.drop(columns=['ID']))

In [ ]:
pred = pd.DataFrame(data={'HOME_WINS': predictions, 'DRAW': np.zeros(len(predictions), dtype=int), 'AWAY_WINS': [int(not x) for x in predictions]})
pred = pd.concat([test_team['ID'], pred],join='inner',axis=1)
pred

In [ ]:
pred.to_csv('./submission.csv', index=False)